In [19]:
import pandas as pd
noSolutionDF = pd.read_csv('output_nosol - copia 2.csv')
solutionDF = pd.read_csv('output_sol - copia 2.csv')

In [20]:
import numpy as np

noSolutionCount = noSolutionDF.shape[0]
solutionCount = solutionDF.shape[0]

print(noSolutionCount)
print(solutionCount)

noSolutionLabel = [0] * noSolutionCount 
solutionLabel = [1] * solutionCount 
Y = noSolutionLabel + solutionLabel
Y = np.array(Y)

1
1


In [21]:
boards = []

for indice, fila in noSolutionDF.iterrows():
    tableros_en_celda = fila['Board'].replace("\r", "").split('\n')
    tableros_en_celda = list(filter(lambda x: x != '', tableros_en_celda))
    tablero_matriz = [list(fila) for fila in tableros_en_celda]
    boards.append(tablero_matriz)
  
    
print("El tamaño de la lista `boards` es:", len(boards))
for indice, fila in solutionDF.iterrows():
    tableros_en_celda = fila['Board'].replace("\r", "").split('\n')
    tableros_en_celda = list(filter(lambda x: x != '', tableros_en_celda))
    tablero_matriz = [list(fila) for fila in tableros_en_celda]
    boards.append(tablero_matriz)

El tamaño de la lista `boards` es: 1


In [22]:

def Rotate_90(tablero):
    return np.rot90(tablero, k=1)

def Rotate_180(tablero):
    return np.rot90(tablero, k=2)

def Rotate_270(tablero):
    return np.rot90(tablero, k=3)

def FlipX(tablero):
    return np.fliplr(tablero)

def FlipY(tablero):
    return np.flipud(tablero)
# Data augmentation
aug_boards = []
for board in boards:
    board_np = np.array(board)
    aug_boards.append(board_np)  
    aug_boards.append(Rotate_90(board_np))  
    aug_boards.append(Rotate_180(board_np)) 
    aug_boards.append(Rotate_270(board_np))  
    aug_boards.append(FlipX(board_np))  
    aug_boards.append(FlipY(board_np))  

print("El tamaño de la lista `boards` es:", len(aug_boards))
with np.printoptions(threshold=np.inf):
    print(aug_boards[3])

El tamaño de la lista `boards` es: 12
[['#' '#' '#' '#' '#']
 ['#' ' ' ' ' '@' '#']
 ['#' ' ' ' ' ' ' '#']
 ['#' '#' '#' '#' '#']
 ['#' '#' '#' '#' '#']]


In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_filas = max(len(tablero) for tablero in aug_boards)
max_columnas = max(max(len(fila) for fila in tablero) for tablero in aug_boards)



tableros_padeados = []

for tablero in aug_boards:
    tablero_padeado = []
    for fila in tablero:
        fila_padeada = list(fila) + ['#'] * (max_columnas - len(fila))
        tablero_padeado.append(fila_padeada)
    while len(tablero_padeado) < max_filas:
        tablero_padeado.append(['#'] * max_columnas)
    tableros_padeados.append(tablero_padeado)


tableros_padeados = np.array(tableros_padeados, dtype=object)


padded_tableros = pad_sequences(tableros_padeados, 
                                maxlen=max_filas, 
                                dtype=object, 
                                padding='post', 
                                value='#')

'''for tablero in padded_tableros:
    for fila in tablero:
        with np.printoptions(threshold=np.inf):
            print(' '.join(fila))
    print()'''

"for tablero in padded_tableros:\n    for fila in tablero:\n        with np.printoptions(threshold=np.inf):\n            print(' '.join(fila))\n    print()"

In [25]:
import numpy as np

def tablero_a_tensor(tablero):
    alto = len(tablero)
    ancho = len(tablero[0])
    tensor = np.zeros((alto, ancho, 7))

    for i in range(alto):
        for j in range(ancho):
            elemento = tablero[i][j]
            if elemento == '#':
                tensor[i, j, 0] = 1
            elif elemento == '$':
                tensor[i, j, 1] = 1
            elif elemento == '.':
                tensor[i, j, 2] = 1
            elif elemento == '@':
                tensor[i, j, 3] = 1
            elif elemento == '*':
                tensor[i, j, 4] = 1
            elif elemento == '+':
                tensor[i, j, 5] = 1
            elif elemento == ' ':
                tensor[i, j, 6] = 1
                
    return tensor


X = np.array([tablero_a_tensor(tablero) for tablero in padded_tableros])
with np.printoptions(threshold=np.inf):
    print(X)
        


[[[[1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 1. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 1.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 1.]
   [0. 0. 0. 0. 0. 0. 1.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 1.]
   [0. 0. 0. 0. 0. 0. 1.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]]]


 [[[1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]
   [1. 0. 0. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 1.]
   [0. 0. 0. 0. 0. 0. 